<a href="https://colab.research.google.com/github/ayseakbaba/Recommender_TOP_N_Movie_Using_Surprise/blob/main/Recommender_TOP_N_Movie_Using_Surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Öneri Sistemlerine Giriş

Öneri sistemleri, sizin sistemde yapmış olduğunuz aksiyonlara bağlı olarak sistem tarafından bir kullanıcı profilinizin oluşmasını ve bu profile uygun beğenebileceğiniz içerikleri sizlerin önüne getirmeyi sağlayan bir sistemdir özetle. Bu çalışma defterimde öneri sistemlerinin son dönemlerde en popüler konularından biri olan en iyi N adet içeriği bir araya getiren bir TOP-N üretmeyi hedefliyorum.

Bunun için Python dilinde çeşitli kütüphaneler bulunmaktadır. Bu kütüphanelerden biri Surprise'dır. Bugünkü projemizde bunu kullanacağız. Bu kütüphane pandas ya da numpy gibi direkt import edilememektedir. Bunun için öncelikle pip komutunu kullanarak bu kütüphaneyi Colab' a eklemeniz gerekmektedir.

In [34]:
!pip install scikit-surprise

Kütüphaneyi ekledikten sonra proje boyunca ihtiyacımız olacak olan kütüphane ve metotları import ediyoruz.

In [35]:
import os
import pandas as pd
from surprise import Dataset, SVD, SVDpp, NMF, SlopeOne
from surprise import accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict

Kütüphane eklendikten sonra bu kütüphanenin bizlere sağlamış olduğu bir kolaylık olarak Datasetlerden yararlanıyoruz. Bu projede filmlerle ilgili yapılan öneri sistemlerinde en sık kullanılan MovieLens 100k verisetini kullanacağım.

In [36]:
data = Dataset.load_builtin('ml-100k')


Yapmış olduğumuz önerilerde filmin id'si yerine direkt filmin adını verebilmek için bu verisetinin içinde yer alan u.item dosyasını kullanmam gerekiyor. Bunun için de hangi klasörde olduğunu bulmaya çalıştım aşağıda.

In [37]:
data_folder = os.path.expanduser('~/.surprise_data/ml-100k')
print(f"Dataset ml-100k is stored in: {data_folder}")

Dataset ml-100k is stored in: /root/.surprise_data/ml-100k


In [38]:
%cd /root/.surprise_data/ml-100k

/root/.surprise_data/ml-100k


In [39]:
%cd ml-100k

/root/.surprise_data/ml-100k/ml-100k


In [40]:
%ls

allbut.pl  u1.base  u2.test  u4.base  u5.test  ub.base  u.genre  u.occupation
mku.sh     u1.test  u3.base  u4.test  ua.base  ub.test  u.info   u.user
README     u2.base  u3.test  u5.base  ua.test  u.data   u.item


u.item dosyasının yerini bulduktan sonra bu '|' işaretine göre split edip ilk iki sütunu alıyoruz. Bu sutünlar movieId ve movie_title değerlerini içermektedir.

In [41]:
data_folder = os.path.expanduser('/root/.surprise_data/ml-100k/ml-100k/')

# Film bilgilerini içeren dosyayı pandas ile yükleyin
movie_info_path = os.path.join(data_folder, 'u.item')
movie_info = pd.read_csv(movie_info_path, sep='|', encoding='latin-1', header=None)
movie_info.columns = ['movie_id', 'movie_title'] + [str(i) for i in range(22)]
movie_dict = dict(zip(movie_info['movie_id'], movie_info['movie_title']))

İhtiyacımız olan bilgileri aldıktan sonra verisetini trainset ve testset olmak üzere %80 ve %20 oranlarında ikiye ayırıyoruz.

In [42]:
trainset, testset = train_test_split(data, test_size=0.2)

Bu örnekte birçok farklı algoritmayı kullanmak ve bunların çıktı değerlerini karşılaştırmak istedim. Yine bu algoritmalar surprise kütüphanesinde mevcut. Bu sayede direkt çağırabiliyoruz.

In [43]:
algo1 = SVD()
algo2 = SVDpp()
algo3 = NMF()
algo4 = SlopeOne()

.fit() metodunu kullanarak ilgili algoritmaların trainsets üzerinde eğitimlerini başlatıyoruz.

In [44]:
algo1.fit(trainset)
algo2.fit(trainset)
algo3.fit(trainset)
algo4.fit(trainset)

Eğitim sonrası algoritmalarımı kullanarak TOP-N önerebilmek için get_recommendations adında bir metot oluşturuyoruz.

In [45]:
def get_recommendations(algo, user_id, item_ids, movie_dict, top_n=5):
    predictions = [algo.predict(user_id, iid) for iid in item_ids]
    predictions.sort(key=lambda x: x.est, reverse=True)
    top_predictions = predictions[:top_n]
    return [(movie_dict[int(pred.iid)], int(pred.est)) for pred in top_predictions] #u.item' i burda kullanabilmek adına yukarıda kullanmıştık

Şimdi bütün bu çalışmaları test edebilmemiz için aşağıdaki kodu yazıyoruz. Bu kodda user_id hangi kullanıcı için TOP-N üretilmesi isteniliyorsa onun id'si iken item_ids'deki döngü testset'teki her bir öğe (uid, iid, rating) üzerinde gezinir. Ancak, burada rating değişkeni kullanılmadığı için bu kısım _ ile gösterilmiştir. _ konvansiyonel olarak bu değerin kullanılmayacağını ifade eder.


In [46]:
user_id = str(196)
item_ids = [iid for (uid, iid, _) in testset if uid == user_id]  # Kullanıcının izleyebileceği filmleri seç

Bu aşamada ise yukarıda tanımlamış olduğumuz algoritmalara uygun olacak şekilde N sayıda film önerisi yapmaktadır. Eğer parametre olarak kaç filmin önerilmesi istendiği girilmezse metotta default olarak 5 ayarlanmıştır. 5 adet film önerecektir.

In [47]:
print("SVD Algoritmasına Göre Öneriler:")
print(get_recommendations(algo1, user_id, item_ids, movie_dict))

print("\nSVD++ Algoritmasına Göre Öneriler:")
print(get_recommendations(algo2, user_id, item_ids, movie_dict))

print("\nNMF Algoritmasına Göre Öneriler:")
print(get_recommendations(algo3, user_id, item_ids, movie_dict))

print("\nSlopeOne Algoritmasına Göre Öneriler:")
print(get_recommendations(algo4, user_id, item_ids, movie_dict))


SVD Algoritmasına Göre Öneriler:
[('Raising Arizona (1987)', 4), ('Men in Black (1997)', 4), ('Adventures of Priscilla, Queen of the Desert, The (1994)', 3), ('That Thing You Do! (1996)', 3), ('Up in Smoke (1978)', 3)]

SVD++ Algoritmasına Göre Öneriler:
[('Adventures of Priscilla, Queen of the Desert, The (1994)', 3), ('Men in Black (1997)', 3), ('That Thing You Do! (1996)', 3), ('Raising Arizona (1987)', 3), ('Mighty Aphrodite (1995)', 3)]

NMF Algoritmasına Göre Öneriler:
[('Raising Arizona (1987)', 4), ('That Thing You Do! (1996)', 3), ('Adventures of Priscilla, Queen of the Desert, The (1994)', 3), ('Men in Black (1997)', 3), ('Mighty Aphrodite (1995)', 3)]

SlopeOne Algoritmasına Göre Öneriler:
[('Raising Arizona (1987)', 3), ('Men in Black (1997)', 3), ('Mighty Aphrodite (1995)', 3), ('Adventures of Priscilla, Queen of the Desert, The (1994)', 3), ('That Thing You Do! (1996)', 3)]


#ÇALIŞMA PERFORMANSI

#RMSE ve MAE Hesaplamaları

In [48]:

def evaluate_algorithms(algorithms, testset):
    metrics = []
    for algo in algorithms:
        predictions = algo.test(testset)
        rmse = accuracy.rmse(predictions, verbose=True)
        mae = accuracy.mae(predictions, verbose=True)
        metrics.append([algo.__class__.__name__, rmse, mae])
    return metrics

algorithms = [algo1, algo2, algo3, algo4]
metrics = evaluate_algorithms(algorithms, testset)



RMSE: 0.9353
MAE:  0.7384
RMSE: 0.9162
MAE:  0.7199
RMSE: 0.9594
MAE:  0.7520
RMSE: 0.9411
MAE:  0.7399


# Precision ve Recall hesaplamaları (k = 5)

In [49]:

def precision_recall_at_k(predictions, k=5, threshold=3.5):
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls


In [50]:
def evaluate_precision_recall(algorithms, testset, k=5):
    precision_recall_metrics = []
    for algo in algorithms:
        predictions = algo.test(testset)
        precisions, recalls = precision_recall_at_k(predictions, k)

        precision = sum(prec for prec in precisions.values()) / len(precisions)
        recall = sum(rec for rec in recalls.values()) / len(recalls)

        precision_recall_metrics.append([algo.__class__.__name__, precision, recall])
    return precision_recall_metrics

precision_recall_metrics = evaluate_precision_recall(algorithms, testset)


# Tüm Metrikleri Bir Tabloda Gösterme

In [51]:
metrics_df = pd.DataFrame(metrics, columns=['Algorithm', 'RMSE', 'MAE'])
precision_recall_df = pd.DataFrame(precision_recall_metrics, columns=['Algorithm', 'Precision@5', 'Recall@5'])

final_metrics_df = pd.merge(metrics_df, precision_recall_df, on='Algorithm')
print("\nAlgoritma Performans Metrikleri:")
print(final_metrics_df)


Algoritma Performans Metrikleri:
  Algorithm      RMSE       MAE  Precision@5  Recall@5
0       SVD  0.935251  0.738380     0.773383  0.439998
1     SVDpp  0.916180  0.719930     0.792718  0.443692
2       NMF  0.959387  0.751968     0.763503  0.423662
3  SlopeOne  0.941119  0.739920     0.776281  0.438846
